# Sentence Selection

Holy Lovenia / 13515113

In [1]:
from nltk import RegexpTokenizer
from pandas.io.json import json_normalize


import json
import pandas as pd
import re

### Read original data

In [2]:
data = []
with open('../dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
                
            data.append(paragraph)

### Preprocess data

In [3]:
preprocessed_data = []
with open('../dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
            
            # replace all dictionary phonetic with ''
            paragraph = re.sub('\/.*\ˈ.*\/', '', paragraph)
            
            # replace all japanese characters with ''
            paragraph = re.sub('[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]+', '', paragraph)
            
            # replace dots in the center of words
            words = paragraph.split(' ')
            for i in range(len(words)):
                if(words[i].find('.') != len(words[i]) - 1 and words[i].find('.') != -1):
                    words[i] = words[i].replace('.', '')
                if(words[i].find(',') != len(words[i]) - 1 and words[i].find(',') != -1):
                    words[i] = words[i].replace(',', '')

            paragraph = ' '.join(words)
            
            data_i_j = paragraph.split('.')
            
            paragraph = []
            for k in range(len(data_i_j)):
                tokenizer = RegexpTokenizer('[\w\/\&\-\:]+', flags=re.UNICODE)
    
                token_list = tokenizer.tokenize(data_i_j[k])
                token_list = [token for token in token_list if len(token) > 1 or token.lower() == 'a']
            
                if token_list != []:
                    paragraph.append(token_list)
                
            preprocessed_data.append(paragraph)

### TextRank

In [4]:
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from operator import itemgetter

import numpy as np

#### Calculate sentence similarity based on `1 - cosine distance`

In [5]:
def sentence_similarity(sentence1, sentence2, stopwords=None):
    """
    sentence1 and sentence2 = ['word1', 'word2', ...]
    """
    
    if stopwords is None:
        stopwords = []
        
    sentence1 = [word.lower() for word in sentence1]
    sentence2 = [word.lower() for word in sentence2]
    all_words = list(set(sentence1 + sentence2))
    
    vector1 = [0] * len(all_words)
    vector2 = vector1.copy()
    
    # 1st sentence's vector building
    for word in sentence1:
        if word in stopwords:
            continue # do nothing
        
        # add 1 to vector's index corresponding to word
        vector1[all_words.index(word)] += 1
    
    # 2nd sentence's vector building
    for word in sentence2:
        if word in stopwords:
            continue # do nothing
        
        # add 1 to vector's index corresponding to word
        vector2[all_words.index(word)] += 1
            
    return 1 - cosine_distance(vector1, vector2)

#### Build similarity matrix for sentences in a paragraph

In [6]:
def sentence_similarity_matrix(sentences, stopwords=None):
    """
    sentences = [['word1, 'word2', ...], [...], ...]
    """
    
    # create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue # do nothing
                
            similarity_matrix[i][j] = sentence_similarity(sentences[i], sentences[j], stopwords)
            
    # row-wise matrix normalization to penalize longer sentences
    for i in range(len(similarity_matrix)):
        similarity_matrix[i] /= similarity_matrix[i].sum()

    return similarity_matrix

#### [PageRank algorithm](https://en.wikipedia.org/wiki/PageRank#Python)

In [7]:
def page_rank(M, eps=0.0001, d=0.85):
    """
    similarity_matrix[i][j] = probability of transitioning from sentence i to sentence j
    eps = stop the algorithm when the difference between two consecutive iterations <= eps
    d (damping factor) = with a probability 1-d the user will simply pick a sentence at random as the next destination, ignoring the structure completely    
    """
    
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    last_v = np.ones((N, 1), dtype=np.float32) * 100
    M_hat = (d * M) + (((1 - d) / N) * np.ones((N, N), dtype=np.float32))

    while np.linalg.norm(v - last_v, 2) > eps:
        last_v = v
        v = np.matmul(M_hat, v)
    
    return v

#### [TextRank](https://medium.com/@aneesha/beyond-bag-of-words-using-pytextrank-to-find-phrases-and-summarize-text-f736fa3773c5)

In [8]:
def rank_text(sentences, top=2, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    
    similarity_matrix = sentence_similarity_matrix(sentences, stopwords)
    
    sentence_ranks = page_rank(similarity_matrix)
    
    # sort sentence ranks
    ranked_sentence_indices = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = ranked_sentence_indices[:top]
    
    return selected_sentences

### Multi-word Phrase Extraction

In [9]:
from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

#### Add POS tags to words

In [10]:
def add_postags(paragraph, stopwords=None):
    postagger = StanfordPOSTaggerWrapper()
    
    postagged_paragraph = []
    for sentence in paragraph:
        postagged_sentence = []
        for word in sentence:
            if word in stopwords:
                continue # skip
            
            postagged_word = postagger.tag(word)
            postagged_sentence.append(postagged_word[0])
        postagged_paragraph.append(postagged_sentence)
    
    return postagged_paragraph

#### Filter words that are nouns or adjectives

In [11]:
def filter_words(postagged_words, tags=['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJS', 'JJR']):
    postag_index = 1
    filtered_words = []
    
    for word in postagged_words:
        if word[postag_index] in tags:
            filtered_words.append(word)
    
    return filtered_words

#### Create one-level array from nested arrays

In [12]:
def flatten_nested_arrays(nested_arrays):
    return [item for sublist in nested_arrays for item in sublist]

#### Find all positions of a certain element in array

In [13]:
def find_all(element, array):
    indices = [i for i, x in enumerate(array) if x == element]
    return indices

#### Check if two words occur together between a pre-defined window size

In [14]:
def do_they_occur_together(word1, word2, flattened_sentences, window_size=3):
    indices1 = find_all(word1, flattened_sentences)
    indices2 = find_all(word2, flattened_sentences)
    
    found = False
    for idx1, idx2 in zip(indices1, indices2):
        if abs(idx1 - idx2) <= window_size:
            found = True
            break
            
    return found    

#### Create occurence matrix

In [15]:
def get_cooccurrence_matrix(all_words, flattened_sentences, window_size=3):
    cooccurrence_matrix = np.zeros((len(flattened_sentences), len(flattened_sentences)))
    
    for i in range(len(all_words)):
        for j in range(len(all_words)):
            if(i == j):
                continue # do nothing
            
            if do_they_occur_together(all_words[i], all_words[j], flattened_sentences, window_size=window_size):
                cooccurrence_matrix[i][j] = 1
                
    return cooccurrence_matrix

#### Pair top N keywords according to occurrence matrix

In [16]:
def pair_keywords(keywords, all_words, cooccurrence_matrix):
    word_index = 0   
    phrases = []
    
    for i in range(len(keywords)):
        for j in range(i + 1):
            if i == j:
                continue # do nothing
                
            idx1 = all_words.index(keywords[i])
            idx2 = all_words.index(keywords[j])
            
            if cooccurrence_matrix[idx1][idx2] == 1:
                phrases.append(all_words[min(idx1, idx2)][word_index] + ' ' + all_words[max(idx1, idx2)][word_index])
                break

    return phrases

#### [Multi-word phrase extraction](https://medium.com/@aneesha/beyond-bag-of-words-using-pytextrank-to-find-phrases-and-summarize-text-f736fa3773c5)

In [17]:
def multi_word_phrase_extraction(sentences, top=5, stopwords=None, window_size=3):
    
    if stopwords is None:
        stopwords = []
    
    # annotate with POS tags
    postagged_sentences = add_postags(sentences, stopwords=stopwords)
    
    flattened_sentences = flatten_nested_arrays(postagged_sentences)
    all_words = list(set(flattened_sentences))
    filtered_words = filter_words(all_words)
    
    cooccurrence_matrix = get_cooccurrence_matrix(filtered_words, flattened_sentences, window_size=window_size)
    
    keyword_ranks = page_rank(cooccurrence_matrix)
    
    # sort keyword ranks
    ranked_keyword_indices = [item[0] for item in sorted(enumerate(keyword_ranks), key=lambda item: -item[1])]
    selected_keywords = ranked_keyword_indices[:top]
    
    keywords = itemgetter(*selected_keywords)(filtered_words)

    phrases = pair_keywords(keywords, filtered_words, cooccurrence_matrix)

    return phrases

#### Get important sentences based on multi-word phrases

In [18]:
def get_important_sentences(multi_word_phrases, preprocessed_sentences, min_occurrence=2):
    all_single_words = []
    for i in range(len(multi_word_phrases)):
        splitted_phrase = multi_word_phrases[i].split()
        all_single_words.append(splitted_phrase)
    
    all_single_words = flatten_nested_arrays(all_single_words)
    all_single_words = list(set(all_single_words))
    
    sentence_scores = np.zeros((len(preprocessed_sentences), 1))
    for i in range(len(preprocessed_sentences)):
        for j in range(len(all_single_words)):
            if all_single_words[j] in preprocessed_sentences[i]:
                sentence_scores[i] = sentence_scores[i] + 1
    
    important_sentence_indices = [i for i, v in enumerate(sentence_scores) if v >= min_occurrence]
    
    return important_sentence_indices

### Usage example

In [19]:
count_same_results = 0

for i in range(len(preprocessed_data)):
    phrases = multi_word_phrase_extraction(preprocessed_data[i])
    multiword_result = get_important_sentences(phrases, preprocessed_data[i])
    rank_text_result = rank_text(preprocessed_data[i])
    
    if (multiword_result == rank_text_result) or (set(multiword_result).issubset(set(rank_text_result))) or (set(rank_text_result).issubset(set(multiword_result))):
        count_same_results = count_same_results + 1
        same = True
    else:
        same = False
    
    print(i, same, multiword_result, rank_text_result)
    print(i, phrases)
    
print('count same results: {} out of {}'.format(count_same_results, len(preprocessed_data)))

0 False [0] [1, 2]
0 ['producer songwriter', 'songwriter American', 'producer singer', 'producer record']
1 True [0] [2, 0]
1 ['pronunciation Chopin', 'pronunciation fʁɑ']


/home/holy/anaconda3/envs/nlp_text/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


2 True [2] [3, 2]
2 ['titles Tibetan', 'acceptance leaders', 'titles various']
3 False [0] [3, 1]
3 ['players pocket', 'players portable']
4 True [0] [3, 0]
4 ['Princess Zelda', 'Princess Japanese', 'Densetsu Zeruda']
5 False [2] [1, 3]
5 ['Wade Purvis', 'John Logan', 'Wade Neal']
6 True [0] [0]
6 ['Standard epicenter', 'Standard PM', 'Standard Time', 'Standard China']
7 True [1] [0, 1]
7 ['art fashion', 'art media', 'art research', 'art finance']
8 True [1] [1, 0]
8 ['American literature', 'American modern', 'American plot', 'American classic']
9 True [0] [0]
9 ['such heating', 'such technologies', 'such solar', 'heating photovoltaics']
10 True [0] [0, 1]
10 ['producer artist', 'entrepreneur fashion', 'entrepreneur designer']
11 True [0] [1, 0]
11 ['धर dharma', 'Pali dharma', 'Sanskrit dharma', 'philosophy dharma']
12 True [3] [2, 3]
12 ['Scotty Phillip', 'Scotty Allen', 'Phillip Candice', 'Scotty McCreery']
13 True [0] [0]
13 ['various capabilities', 'various sensory', 'various behav

90 False [0, 1] [1, 2]
90 ['Somalis Arabic', 'majority Peninsula', 'Somalis Somali']
91 True [0, 2, 3] [2, 0]
91 ['Ages medieval', 'Ages Middle', 'Ages history', 'European Middle']
92 False [1] [2, 0]
92 ['features mora', 'features articulatory', 'articulatory rime', 'rime syllable']
93 True [0, 1] [1, 0]
93 ['element central', 'CPU unit', 'element least']
94 True [2] [2, 1]
94 ['many communities', 'many individuals', 'black communities', 'many context']
95 True [1] [2, 1]
95 ['title Daily', 'Daily Register', 'title Universal']
96 True [0, 1] [0, 1]
96 ['seat government', 'India government', 'seat capital', 'addition India']
97 False [3] [2, 1]
97 ['natural barriers', 'natural routes', 'such natural', 'natural specific']
98 False [1, 2] [1, 0]
98 ['Galloway Harbor', 'city Brigantine', 'Township Harbor']
99 False [2] [1, 0]
99 ['bacteriology transplantation', 'bacteriology virology', 'bacteriology psychiatry', 'transplantation organ']
100 True [0] [1, 0]
100 ['Audio MPEG-2', 'Audio Laye

179 False [2] [3, 1]
179 ['architects John', 'Blore Edward', 'Blore Nash']
180 False [2] [1, 4]
180 ['redeposits metal', 'redeposits process', 'vapor redeposits', 'redeposits chemical']
181 False [6] [5, 0]
181 ['league position', 'league average', 'league highest', 'century highest']
182 False [1] [0, 3]
182 ['insect rash-causing', 'rash-causing rough', 'insect plants', 'insect surfaces']
183 True [0, 1] [1, 0]
183 ['team American', 'team professional', 'team baseball', 'located team']
184 False [0] [3, 2]
184 ['조국해방전쟁 Chosungul', 'Fatherland Haebang', 'Fatherland Liberation']
185 False [0] [1, 2]
185 ['certain exclusive', 'certain rights', 'certain permission']
186 True [1] [1, 2]
186 ['land shares', 'land peninsula', 'borders land', 'shares Balkan']
187 True [0] [2, 0]
187 ['trading generation', 'trading power', 'distribution refining']
188 True [0] [1, 0]
188 ['such elephants', 'such primates', 'elephants humans', 'such intelligent']
189 False [0] [1, 2]
189 ['Queen permission', 'Q

269 False [2, 3] [3, 1]
269 ['Scandinavian trade', 'trade ecclesiastical', 'Scandinavian global', 'Scandinavian network']
270 False [3] [1, 2]
270 ['Islands Cortegada', 'Islands Ons', 'Islands Sálvora', 'Islands Cíes']
271 True [0] [1, 0]
271 ['players disk', 'players drives', 'players portable']
272 True [0] [0, 1]
272 ['North Sichuan', 'North Zhou', 'North dynasties', 'North China']
273 True [2] [3, 2]
273 ['such properties', 'properties normalization', 'properties rules', 'normalization decomposition']
274 False [0, 2, 4] [4, 1]
274 ['border Canada', 'border United', 'border States', 'area United']
275 True [5] [5, 3]
275 ['Greater forms', 'Greater note', 'Greater conurbation', 'bulk conurbation']
276 True [0] [0]
276 ['Management Theory', 'Management Terror', 'posits Management', 'time Management']
277 False [0, 6] [5, 6]
277 ['Arabic Sahara', 'Desert al-kubrā', 'Arabic aṣ-ṣaḥrāʾ']
278 False [0, 1] [3, 5]
278 ['individual arbitrary', 'individual officials', 'individual government',

357 True [2] [2, 0]
357 ['Churchill Winston', 'Churchill Minister', 'Churchill retaliatory', 'Churchill Prime']
358 True [] [1, 0]
358 []
359 False [1, 5] [5, 4]
359 ['adjective vacuus', 'adjective Latin', 'term Latin', 'adjective vacant']
360 False [0] [3, 4]
360 ['cháo Hàn', 'cháo pinyin', 'Hàn dynasty']
361 False [1] [0, 2]
361 ['messages series', 'messages divine', 'culmination series']
362 True [2] [2, 0]
362 ['Statistics Nations', 'Statistics United', 'Statistics Division']
363 False [0, 1] [0, 3]
363 ['digital disc', 'digital optical', 'disc format', 'digital data']
364 False [0] [1, 3]
364 ['signals electronic', 'signals electrical', 'power signals', 'signals switch']
365 False [0] [3, 1]
365 ['many people', 'people era', 'people sense', 'people Pre-Modern']
366 True [1] [1, 2]
366 ['military American', 'military cultural', 'military influence', 'military control']
367 False [0, 2] [3, 1]
367 ['power converts', 'power radio', 'power electric', 'converts device']
368 False [3] [

In [20]:
idx = 121

for i in range(len(preprocessed_data[idx])):
    print(i, ' '.join(preprocessed_data[idx][i]))

0 Emotions are complex
1 According to some theories they are a state of feeling that results in physical and psychological changes that influence our behavior
2 The physiology of emotion is closely linked to arousal of the nervous system with various states and strengths of arousal relating apparently to particular emotions
3 Emotion is also linked to behavioral tendency
4 Extroverted people are more likely to be social and express their emotions while introverted people are more likely to be more socially withdrawn and conceal their emotions
5 Emotion is often the driving force behind motivation positive or negative
6 Definition has been described as is a positive or negative experience that is associated with a particular pattern of physiological activity According to other theories emotions are not causal forces but simply syndromes of components which might include motivation feeling behavior and physiological changes but no one of these components is the emotion
7 Nor is the emoti